In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import col, to_date

In [4]:
spark = SparkSession.builder \
    .master("spark://localhost:7077") \
    .appName('test') \
    .getOrCreate()

24/03/03 21:12:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
df = spark.read \
    .parquet('fhv_trip/2019/10/')

24/03/03 21:12:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/03 21:13:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/03 21:13:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/03 21:13:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/03 21:13:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
df.printSchema()

In [109]:
# To filter on 2019-10-15
df = df.withColumn("pickup_date",to_date("pickup_datetime"))
filterred_trip = df.filter(col("pickup_date") == "2019-10-15")
filterred_trip.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+
|              B03162|2019-10-15 09:34:17|2019-10-15 10:16:32|          69|          32|   NULL|                B03162| 2019-10-15|
|              B02182|2019-10-15 11:45:48|2019-10-15 12:33:24|         264|         264|   NULL|                B02835| 2019-10-15|
|              B00256|2019-10-15 09:41:20|2019-10-15 10:18:31|         264|         264|   NULL|                B00256| 2019-10-15|
|              B00706|2019-10-15 17:30:17|2019-10-15 17:53:42|         245|         221|   NULL|                B00706| 2019-10-15|
|              B00457|2019-10-15 05:09:51|2019-10-15 05:18:50|         264| 

In [118]:
# Find the longest trip
df = df.withColumn("duration_seconds", (col("dropOff_datetime").cast("long") - col("pickup_datetime").cast("long")))
longest_trip = df.orderBy(col("duration_seconds").desc()).first()

duration_seconds = longest_trip["duration_seconds"]
duration_hours = duration_seconds / 3600

631152.5


In [127]:
# Least frequent pickup location zone
zone_df = spark.read.csv('taxi_zone_lookup.csv', header=True, inferSchema=True)

pickup_counts_df = df.groupBy("PUlocationID").count()

result_df = pickup_counts_df.join(zone_df, pickup_counts_df["PUlocationID"] == zone_df["LocationID"])

least_frequent_zone = result_df.orderBy(col("count")).first()

# Display the result
print("Least frequent pickup location zone:")
least_frequent_zone

Least frequent pickup location zone:


Row(PUlocationID=2, count=1, LocationID=2, Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone')